In [ ]:
import spiceypy as spice
import math
from datetime import datetime, timedelta

def eq_to_ho(time, dec, ra, lat, lon):
	# hour angle
	lst = local_sidereal_time(time, lon)
	ha = math.radians(lst) - ra
	
	
	#change everything to radians
	lon = math.radians(lon)
	lat = math.radians(lat)
	
	# compute azimuth and alitude from lat, lon, declination, and hour angle
	x = math.cos(ha) * math.cos(dec)
	y = math.sin(ha) * math.cos(dec)
	z = math.sin(dec)

	xhor = x * math.sin(lat) - z * math.cos(lat)
	yhor = y
	zhor = x * math.cos(lat) + z * math.sin(lat)
	
	azm  = math.atan2(yhor, xhor) + math.radians(180)
	alt = math.atan2(zhor, math.sqrt(xhor*xhor+yhor*yhor))
	
	#convert back to degrees
	alt = math.degrees(alt)
	azm = math.degrees(azm)
	
	return rev(alt), rev(azm)
	
def local_sidereal_time(time, lon):
	year = time.year
	month = time.month
	day = time.day
	hours = time.hour
	minutes = time.minute
	seconds = time.second
	#get current seconds since J2000
	et = spice.str2et(str(time))
	
	# get the sun's ecliptic longitude. The sun's right ascension if the Earth is the observer
	sun_position, lightTimes = spice.spkpos("EARTH", et, "J2000", "NONE", "SUN")
	sun_range, sun_ra, sun_dec = spice.recrad(sun_position)
	
	utc_local_time_difference = 0
	
	#convert from radians to degrees
	gmst0 = math.degrees(sun_ra)
	utc_hours = hours + (minutes / 60.0 + ((seconds / 60.0) / 60.0))
	utc_hours = utc_hours - utc_local_time_difference
	#Get gmst by converting hours to degrees and adding to gmst at 0h
	gmst = gmst0 + (utc_hours * 15)
	lst = gmst + lon
	
	return rev(lst)
	
# Return an angle between 0 and 360 degrees or 0 and -360 degrees
def rev(angle):
	if angle < 0:
		return (angle % 360) - 360
	else:
		return angle % 360

def daterange(start_date, end_date):
	for n in range(int ((end_date - start_date).days)):
		yield start_date + timedelta(n)

spice.furnsh('./kernel/MetaK.txt')

targ = "MOON"
ref = "J2000"
obs = "EARTH"

#Users current lat and lon
lat = 0
lon = 0

start_date = datetime(2019, 8, 12, 17, 39, 0)
end_date = datetime(2019, 8, 13, 17, 39, 0)

for now in daterange(start_date, end_date):

	# get et values
	et = spice.str2et(str(now))
	# run spkpos as a vectorized function
	positions, lightTimes = spice.spkpos(targ, et, ref, "NONE", obs)

	#get right ascension, declination, and distance
	range, ra, dec = spice.recrad(positions)
	
	alt, azm = eq_to_ho(now, dec, ra, lat, lon)
	#if(alt > 15):
	print("date: {}".format(str(now)))
	print("altitude: {} \n azimuth: {}".format(alt, azm))
# clean up the kernels
spice.kclear()







In [ ]:
# This function solves the non-linear problem, we need to use SPICE to completley define all the variables to this problem.
# This will require to also change the function a bit to make the args[] change with time

import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 


# Define the second-order ODE system
def odefunc(t, r, r_moon_earth, r_moon_s_hat, r_earth_s_hat):
    G = 6.67430e-11
    mass_moon = 7.35e22
    mass_earth = 5.97e24
    dxdt = r[1]
    dydt = r[3]
    dzdt = r[5]
    distance_arr = np.array([r[0],r[2],r[4]])
    dx2dt2 = r_moon_s_hat[0]*G*mass_moon/(abs(distance_arr+r_moon_earth))^2 + r_earth_s_hat[0]*G*mass_earth/(abs(distance_arr))^2
    dy2dt2 = r_moon_s_hat[1]*G*mass_moon/(abs(distance_arr+r_moon_earth))^2 + r_earth_s_hat[1]*G*mass_earth/(abs(distance_arr))^2
    dz2dt2 = r_moon_s_hat[2]*G*mass_moon/(abs(distance_arr+r_moon_earth))^2 + r_earth_s_hat[2]*G*mass_earth/(abs(distance_arr))^2
    return [dxdt, dx2dt2, dydt, dy2dt2, dzdt, dz2dt2]


# Set the initial conditions
initial_time = 0
initial_state = [6371, 0, 0, 0, 0, -0.047]  # x(0) = 1, dx/dt(0) = 0, y(0) = 2, dy/dt(0) = 0 ...

# Set the time span for integration
t_span = (0, 24*3600)

# Solve the second-order ODE system using solve_ivp
r_moon_s_hat_earth = []
r_moon_s_hat = []
r_earth_s_hat = []
sol = solve_ivp(odefunc, t_span, initial_state, args=(r_moon_s_hat_earth, r_moon_s_hat, r_earth_s_hat,), t_eval=np.linspace(t_span[0], t_span[1], 100))

# Plot the solutions for x and y
plt.plot(sol.t, sol.y[0], label='x(t)')
plt.plot(sol.t, sol.y[2], label='y(t)')
plt.plot(sol.t, sol.y[4], label='z(t)')
plt.title('Solution of the Second-Order ODE System')
plt.xlabel('Time')
plt.ylabel('Function Values')
plt.legend()
plt.show()

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(sol.y[0], sol.y[2], sol.y[4], label='3D Curve')
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')
ax.set_title('3D Plot Example')
ax.legend()
plt.show()
